In [ ]:
import matplotlib
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm, trange
from zoo import models
from common.data_loader import DataLoader
from util import visualize_model

## Device configuration

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Configuration

In [ ]:
num_epochs = 50
num_classes = 10
batch_size = 256
learning_rate = 0.001
momentum = 0.9


transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Pad(4),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

## Load data

In [ ]:
data_loader = DataLoader()
data_loader.set_transform(transform)
train_data = data_loader.get_train_data(batch_size)
test_data = data_loader.get_test_data(batch_size)

In [ ]:
print(train_data.__dict__)

In [ ]:
import inspect
models_ = []
for name, obj in inspect.getmembers(models):
    if inspect.isclass(obj) and inspect.getmodule(obj).__name__.split(".")[-1] == "models":
        models_.append(obj)
        # print (obj)
# print(models.__dict__)

## Train model

In [ ]:
def train_model(model):
    total_step = len(train_data)
    for epoch in trange(num_epochs):
        for i, (images, labels) in enumerate(train_data):
            # Place training batch onto the appropriate device
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass: compute predicted y by passing x to the model.
            outputs = model(images)

            # Compute loss.
            loss = loss_function(outputs, labels)

            # Before the backward pass, use the optimizer object to zero all of the
            # gradients for the variables it will update (which are the learnable
            # weights of the model). This is because by default, gradients are
            # accumulated in buffers(i.e, not overwritten) whenever .backward()
            # is called. Checkout docs of torch.autograd.backward for more details.
            optimizer.zero_grad()
            # Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Calling the step function on an Optimizer makes an update to its parameters
            optimizer.step()

            if (i + 1) % 100 == 0:
                tqdm.write('\nEpoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                # tqdm.write("\nlol")

        # Save the model checkpoint
        torch.save(model.state_dict(), 'model.ckpt')

## Evaluate model

In [ ]:
def eval_model(model):
    # Test the model
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance; dropout disabled)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_data:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the {} test images: {} %'.format(len(test_data), 100 * correct / total))


    # visualize_model(model, device, test_data, num_images=15,
    #                 class_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    #                 )
    # matplotlib.pyplot.waitforbuttonpress()

## Pipeline

In [ ]:
for model in models_:
    print(model.__name__)
    model = model()
    # print(model)
    model.to(device)

    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_model(model)
    eval_model(model)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

# print(model)

78.66% - baseline